# PyGeM
## Tutorial 6: Deformation of the computational mesh through an RBF interpolator

In this tutorial we're going to show the procedure to propagate a given object deformation to the computational grid built around such a object. In this way, any deformation mapping we can apply to the object to morph is replicated to all discretized space around it, avoiding the expensive re-meshing phase. In this tutorial we just show the needed steps to achieve such parametric mesh exploiting the **PyGeM** package, presenting a very simple test-case, but for a practical real-world usecase we refer to
-  Hull Shape Design Optimization with Parameter Space and Model Reductions, and Self-Learning Mesh Morphing
by Nicola Demo, Marco Tezzele, Andrea Mola and Gianluigi Rozza. J. Mar. Sci. Eng. 2021, 9(2), 185; https://doi.org/10.3390/jmse9020185.


### The numerical settings
The methodology that follows is very general and can be extended to many different scenario, since it basically requires only the coordinates of the nodes of the object geometry and of the (undeformed) initial mesh. For sake of simplicity, here we present the deformation of an [OpenFOAM](https://openfoam.org/) grid for simulating a 2D Navier-Stokes flow around a cylinder. We assume that this cilinder is the object to deform.
Even if the entire procedure is employable also when the deformation mapping applied to the initial object is unknown (we see in few lines that the required input is just the displacement of the initial object after the deformation), here we apply the *free-form deformation* method to the undeformed cylinder in order to parametrize its geometry.

First of all, we import all the libraries which we're going to use:
- `numpy` and `matplotlib` for the generic scientific environment;
- `Smithers` for dealing with the OpenFOAM mesh;
- `PyGeM` for the object and mesh deformation.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from smithers.io.openfoam.openfoamhandler import OpenFoamHandler
from scipy.interpolate import Rbf
from pygem import FFD, RBF

import plotly.graph_objects as go

Then we define the auxiliary function `scatter3d` which we're going to use often to plot several objects as lists of 3D points. You do not need to understand the exact details of this function since we are going to use it only to show the results:

In [202]:
def scatter3d(arr, figsize=(800, 800), s=10, draw=True, alpha=1, labels=None, original_control_points=None, control_points=None):
    fig = go.Figure()

    for idx, a in enumerate(arr):
        trace = go.Scatter3d(x=a[:, 0], y=a[:, 1], z=a[:, 2], mode='markers', marker=dict(size=s, opacity=alpha))
        if labels is not None:
            trace.name = labels[idx]
        fig.add_trace(trace)

    # control point
    if (control_points is not None) and (original_control_points is not None):
        point_trace=go.Scatter3d(
            x=original_control_points[:, 0],
            y=original_control_points[:, 1],
            z=original_control_points[:, 2],
            mode="markers",
            marker=dict(size=s*3, symbol="square", color="gray"),
            name="original_control_points",
        )
        fig.add_trace(point_trace)
        
        point_trace=go.Scatter3d(
            x=control_points[:, 0],
            y=control_points[:, 1],
            z=control_points[:, 2],
            mode="markers",
            marker=dict(size=s*3, symbol="square", color="red"),
            name="deformed_control_points",
        )
        fig.add_trace(point_trace)

        for ol, dl in zip(original_control_points, control_points):
            line_trace = go.Scatter3d(
                x=[ol[0], dl[0]],
                y=[ol[1], dl[1]],
                z=[ol[2], dl[2]],
                mode='lines', 
                line=dict(color='red'),
                showlegend=False,
            )
            fig.add_trace(line_trace)

    fig.update_layout(
        scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), 
        width=figsize[0], 
        height=figsize[1],
    )

    if draw:
        fig.show()
    else:
        return fig


## 1) Extraction of the mesh points

In few words, the procedure uses the the nodes coordinates of the deformed object in order to fit the RBF interpolator which will propagate such morphing to the computational grid.
The first step is then the extraction of such points: we specify that, for avoiding that also the mesh boundaries are deformed, we need to force a *zero* displacement there. We can easily obtain that by passing these bondaries as the RBF control points.

As we mentioned before, in this tutorial we use the library `Smithers` to load the OpenFOAM mesh from the folder `openfoam_mesh` which serves as example. First of all, we use the method `read()` from the class `OpenFoamHandler` to load the data. This method returns a dictionary which contains all the informations available about the mesh, included the list of points (`mesh['points']`).

In [203]:
from smithers.io.obj.objhandler import ObjHandler

In [204]:
objhandler = ObjHandler()
mesh = objhandler.read("bunny/bunny.obj")

skipping: ['#', 'OBJ', 'file', 'format', 'with', 'ext', '.obj']
skipping: ['#', 'vertex', 'count', '=', '2503']
skipping: ['#', 'face', 'count', '=', '4968']


Moreover, the object returned by `read()` contains a list of points for each *boundary*, represented by a list of indexes which refers to `mesh['points']`. We can use these lists to obtain the coordinates of the points which compose the cylinder (which we call *obstacle*) and walls.

In [205]:
obstacle=mesh.vertices  #WavefrontOBJの変数を参照

# 軸を変える場合
#obstacle = obstacle[:, [2,0,1]]

At this point we can plot the obstacle and the walls using the auxiliary function `scatter3d`:

In [206]:
# 間引き用のindex
ratio = 0.2
n_points = len(mesh.vertices)

idx = np.array(range(n_points))
np.random.shuffle(idx)
sample_idx = idx[:int(n_points*ratio)]

As you can see our geometry is made of two faces, one at `z=0.5` and the other at `z=-0.5`.

## 2) Object Deformation

Here we need to apply a generic deformation on the initial object (the cylinder here). In case this deformation is already computed, you can skip this section, having the forethought of storing the nodes coordinates of the undeformed and deformed object.

We use the `FFD` deformation from [PyGeM](https://github.com/mathLab/PyGeM) (for a reference check [this tutorial](http://mathlab.github.io/PyGeM/tutorial-1-ffd.html)) to deform the original object (the upper and lower faces of a cylinder). We create the new `FFD` object and set its attributes in order to create a simple deformation

In [214]:
# tolerance
tol=0.00001

# bounding box
min = obstacle.min(axis=0)
max = obstacle.max(axis=0)
box_origin = min - tol
box_length = abs(min - max) + 2 * tol

min, max

(array([-0.09438042,  0.0333099 , -0.06167917]),
 array([0.0607788 , 0.18699602, 0.05871464]))

In [215]:
ffd = FFD([2, 2, 2])

ffd.box_origin = np.array(box_origin)
ffd.box_length = np.array(box_length)

val =  0.5

ffd.array_mu_z[0, 1, 0] = val
ffd.array_mu_x[0, 1, 0] = -val

ffd.array_mu_z[1, 1, 0] = -val
ffd.array_mu_x[1, 1, 0] = -val

ffd.array_mu_z[0, 1, 1] = val
ffd.array_mu_x[0, 1, 1] = val

ffd.array_mu_z[1, 1, 1] = -val
ffd.array_mu_x[1, 1, 1] = val

ffd.control_points()

array([[-0.09439042,  0.0332999 , -0.06168917],
       [-0.09439042,  0.0332999 ,  0.05872464],
       [-0.01680081,  0.18700602, -0.12189607],
       [-0.17198003,  0.18700602, -0.00148227],
       [ 0.0607888 ,  0.0332999 , -0.06168917],
       [ 0.0607888 ,  0.0332999 ,  0.05872464],
       [ 0.13837841,  0.18700602, -0.00148227],
       [-0.01680081,  0.18700602,  0.11893154]])

We then operate the deformation and plot the result, against the old version of the obstacle.

In [216]:
ffd.control_points(deformed=False)

array([[-0.09439042,  0.0332999 , -0.06168917],
       [-0.09439042,  0.0332999 ,  0.05872464],
       [-0.09439042,  0.18700602, -0.06168917],
       [-0.09439042,  0.18700602,  0.05872464],
       [ 0.0607888 ,  0.0332999 , -0.06168917],
       [ 0.0607888 ,  0.0332999 ,  0.05872464],
       [ 0.0607888 ,  0.18700602, -0.06168917],
       [ 0.0607888 ,  0.18700602,  0.05872464]])

In [217]:
new_obstacle = ffd(obstacle)
scatter3d(
    [new_obstacle[:], obstacle[:]], 
    s=1, labels=['deformed', 'original'], 
    original_control_points=ffd.control_points(deformed=False), control_points=ffd.control_points(deformed=True),
    alpha=0.5
)

In [218]:
# 元のメッシュに上書きし，objファイルを保存

import copy
new_mesh=copy.copy(mesh)
new_mesh.vertices=new_obstacle

objhandler.write(new_mesh, "bunny/deformed.obj")